In [156]:
import pandas as pd
import re
import sys

In [28]:
pd.set_option('display.max_columns', None)

In [413]:
file = "AL-10128988-2367.snp.hf.pass.vcf"
#gen_f = "AL-1324117.snp.hf.pass.vcf"
gen_f = "AL-188505-302.snp.hf.pass.vcf"
bed = "../annotation/genes/repeats_annot.bed"
indel_file ="SOMOZACOL2.combined.hf.vcf"

In [59]:
def calculate_ALT_AD(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    if row.len_AD > 2:
        max_AD = max(split_AD)
        #max_index = split_AD.index(max(split_AD))
        return max_AD
    else:
        #ALT_AD = row.AD.split(",")[1]
        return split_AD[0]

In [60]:
def calculate_AD_list(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    max_AD = max(split_AD)
    row.AD_list = split_AD
    #max_index = split_AD.index(max(split_AD))
    return split_AD

In [407]:
def handle_polymorphism(vcf_df):
    for index, _ in vcf_df[vcf_df.len_AD > 2].iterrows():
        split_AD = vcf_df.loc[index, 'AD'].split(",")[1:]
        split_AD = [int(x) for x in split_AD]
        if min(split_AD) == 0:
            max_index = split_AD.index(max(split_AD)) #Obtain index from highest value in list of positions
            split_ALT = vcf_df.loc[index, 'ALT'].split(",") #split bases into list
            vcf_df.loc[index, 'ALT'] = split_ALT[max_index] #Retrieve the base using ps index
            vcf_df.loc[index, 'len_AD'] = 2 #reset number of alternatives as normal
        elif max(split_AD)/min(split_AD) > 1.5: # the same process to avoid dividing by 0
            max_index = split_AD.index(max(split_AD))
            split_ALT = vcf_df.loc[index, 'ALT'].split(",")
            vcf_df.loc[index, 'ALT'] = split_ALT[max_index]
            vcf_df.loc[index, 'len_AD'] = 2

In [312]:
handle_polymorphism(test_pmf)

In [313]:
test_pmf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,AD_list,gt0,gt1,dp,aF,AF
695,MTB_anc,2626189,.,A,C,101892.61,PASS,"AC=1,1;AF=0.500,0.500;AN=2;BaseQRankSum=0.789;...",GT:AD:DP:GQ:PL,"1/2:0,29,5:34:99:1425,207,120,1218,0,1245",1.0,1.0,0.5,34.0,0.789,34.0,3.0103,0.0,45.58,29.66,1/2,"0,29,5",99.0,"1425,207,120,1218,0,1245",0.5,0.0160,-0.041,NaN,NaN,NaN,2,0.0,29.0,"[29, 5]",1,2,29.0,0.0,1.0
696,MTB_anc,2626191,.,T,C,101887.04,PASS,"AC=1,1;AF=0.500,0.500;AN=2;BaseQRankSum=0.330;...",GT:AD:DP:GQ:PL,"1/2:0,29,5:34:99:1425,207,120,1218,0,1245",1.0,1.0,0.5,34.0,0.330,34.0,3.0103,0.0,45.49,36.86,1/2,"0,29,5",99.0,"1425,207,120,1218,0,1245",0.5,0.0064,-0.041,NaN,NaN,NaN,2,0.0,29.0,"[29, 5]",1,2,29.0,0.0,1.0


In [411]:
def import_VCF42_to_pandas(vcf_file, sep='\t'):
    """
    Script to read vcf 4.2
    - now handle correct allele frequency calculated by summing REF reads + ALT reads instead from DP parameter
    - now retrieve the largest read number for ALT allele frequency in case is a heterozygous SNP (depends on calculate_ALT_AD())
    - now uses dataframe.iterrows() instead dataframe.index
    """
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        
        #Use first line as header
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
        sample = dataframe.columns[-1]
        dataframe.rename(columns={sample:'sample'}, inplace=True)
        
        for index, data_row in dataframe.iterrows():
            info_fields = re.findall(r';*([a-zA-Z]{1,20})=', data_row.INFO)
            info_values = re.findall(r'-?\d+\.?\d*e?[+-]?\d{0,2}', data_row.INFO)
            
            format_fields = data_row['FORMAT'].split(":")
            format_values = data_row['sample'].split(":")
                                    
            for ifield, ivalue in zip(info_fields,info_values):
                dataframe.loc[index,ifield] = ivalue
                
            for ffield, fvalue in zip(format_fields,format_values):
                dataframe.loc[index,ffield] = fvalue
            
            
        dataframe.rename(columns={'AF':'af'}, inplace=True)
        
        dataframe['len_AD'] = dataframe['AD'].str.split(",").str.len()
        dataframe['REF_AD'] = dataframe['AD'].str.split(",").str[0]
        
        dataframe['ALT_AD'] = dataframe.apply(calculate_ALT_AD, axis=1)
        dataframe[['gt0','gt1']] = dataframe['GT'].str.split(r'[/|\|]', expand=True)
        
        # this step remove false snps from cohort calling and reset index
        #dataframe = dataframe[dataframe.ALT_AD > 0].reset_index(drop=True)
        
        handle_polymorphism(dataframe)
        
        #This step removes ALT alleles wich, after filtration, remained the most abundant
        #dataframe = dataframe[dataframe.ALT != '*'].reset_index(drop=True)
        
        
        to_float = ['QUAL', 'AC', 'af', 'AN', 'BaseQRankSum', 'DP', 'ExcessHet', 'FS',
       'MLEAC', 'MLEAF', 'MQ', 'MQRankSum', 'QD', 'ReadPosRankSum', 'SOR','GQ','ALT_AD', 'REF_AD']
        
        to_int = ['POS', 'len_AD', 'gt0', 'gt1']
        
        to_str = ['#CHROM','REF','ALT', 'FILTER']
        
        for column in dataframe.columns:
            if column in to_float:
                dataframe[column] = dataframe[column].astype(float)
                
        for column in dataframe.columns:
            if column in to_int:
                dataframe[column] = dataframe[column].astype(int)
                
        for column in dataframe.columns:
            if column in to_str:
                dataframe[column] = dataframe[column].astype(str)
                
        dataframe['dp'] = (dataframe['REF_AD'] + dataframe['ALT_AD'])
        dataframe['aF'] = dataframe['REF_AD']/dataframe['dp']
        dataframe['AF'] = dataframe['ALT_AD']/dataframe['dp']
             
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe

# Polymorphism

In [364]:
gen = import_VCF42_to_pandas(gen_f)

In [375]:
gen[(gen.ALT_AD < 2) & (gen.AF > 0.75)]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,BaseQRankSum,DP,ExcessHet,FS,InbreedingCoeff,MQ,MQRankSum,QD,ReadPosRankSum,SOR,GT,AD,GQ,PL,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
9,MTB_anc,21819,.,C,A,13071.10,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:79,6,0",2.0,1.0,2.0,NaN,2.0,3.0103,0.000,-0.0217,60.00,NaN,30.02,NaN,1.122,1/1,"0,2",6.0,"79,6,0",NaN,NaN,NaN,2,0.0,2.0,1,1,2.0,0.0,1.0
193,MTB_anc,483292,.,A,T,306.39,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-4.260e-01;DP=1...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:26,3,0",2.0,1.0,2.0,-0.426,1.0,1.9262,1.484,0.0576,60.00,0.0,2.79,-1.938,0.373,1/1,"0,1",3.0,"26,3,0",NaN,NaN,NaN,2,0.0,1.0,1,1,1.0,0.0,1.0
324,MTB_anc,1095944,.,C,T,4617.25,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:9:1|1:1095942_A_G:134,9,0:1095942",2.0,1.0,2.0,NaN,2.0,0.0000,0.000,0.8101,48.95,NaN,30.03,NaN,0.693,1|1,"0,2",9.0,"134,9,0",1|1,1095942_A_G,1095942,2,0.0,2.0,1,1,2.0,0.0,1.0
325,MTB_anc,1095945,.,T,G,4582.85,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:9:1|1:1095942_A_G:134,9,0:1095942",2.0,1.0,2.0,NaN,2.0,0.0000,0.000,0.8029,48.95,NaN,25.90,NaN,0.693,1|1,"0,2",9.0,"134,9,0",1|1,1095942_A_G,1095942,2,0.0,2.0,1,1,2.0,0.0,1.0
353,MTB_anc,1165114,.,T,C,13018.51,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:80,6,0",2.0,1.0,2.0,NaN,2.0,3.0103,0.000,-0.0273,40.00,NaN,30.59,NaN,1.855,1/1,"0,2",6.0,"80,6,0",NaN,NaN,NaN,2,0.0,2.0,1,1,2.0,0.0,1.0
364,MTB_anc,1191376,.,G,C,100.97,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.0663;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:9:1|1:1191358_T_G:126,9,0:1191358",2.0,1.0,2.0,NaN,2.0,0.0663,0.000,0.3561,60.00,NaN,27.66,NaN,2.303,1|1,"0,2",9.0,"126,9,0",1|1,1191358_T_G,1191358,2,0.0,2.0,1,1,2.0,0.0,1.0
365,MTB_anc,1191377,.,T,C,102.75,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.7812;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:9:1|1:1191358_T_G:126,9,0:1191358",2.0,1.0,2.0,NaN,2.0,0.7812,0.000,0.1328,60.00,NaN,36.76,NaN,2.303,1|1,"0,2",9.0,"126,9,0",1|1,1191358_T_G,1191358,2,0.0,2.0,1,1,2.0,0.0,1.0
394,MTB_anc,1276771,.,C,T,6936.44,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:6:1|1:1276771_C_T:90,6,0:1276771",2.0,1.0,2.0,NaN,2.0,0.0000,0.000,0.9744,42.96,NaN,30.33,NaN,0.813,1|1,"0,2",6.0,"90,6,0",1|1,1276771_C_T,1276771,2,0.0,2.0,1,1,2.0,0.0,1.0
395,MTB_anc,1276774,.,T,A,6936.44,PASS,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:6:1|1:1276771_C_T:90,6,0:1276771",2.0,1.0,2.0,NaN,2.0,0.0000,0.000,0.9744,42.83,NaN,35.08,NaN,0.800,1|1,"0,2",6.0,"90,6,0",1|1,1276771_C_T,1276771,2,0.0,2.0,1,1,2.0,0.0,1.0
452,MTB_anc,1488793,.,G,C,2131.28,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.593e+00;DP=2...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,2:2:6:1|1:1488793_G_C:90,6,0:1488793",2.0,1.0,2.0,-1.593,2.0,0.0000,0.000,0.6606,60.00,0.0,30.45,0.282,0.299,1|1,"0,2",6.0,"90,6,0",1|1,1488793_G_C,1488793,2,0.0,2.0,1,1,2.0,0.0,1.0


In [371]:
gen.columns

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'sample', 'AC', 'af', 'AN', 'BaseQRankSum', 'DP', 'ExcessHet', 'FS',
       'InbreedingCoeff', 'MQ', 'MQRankSum', 'QD', 'ReadPosRankSum', 'SOR',
       'GT', 'AD', 'GQ', 'PL', 'PGT', 'PID', 'PS', 'len_AD', 'REF_AD',
       'ALT_AD', 'gt0', 'gt1', 'dp', 'aF', 'AF'],
      dtype='object')

In [343]:
polym_list = gen.POS[gen.len_AD > 2].tolist()

In [317]:
gen[gen.len_AD > 2][['#CHROM', 'POS', 'ID', 'REF', 'ALT','AC', 'af', 'AN', 'DP', 'GT', 'AD', 'GQ',  'len_AD', 'REF_AD', 'ALT_AD',
       'gt0', 'gt1']]

,#CHROM,POS,ID,REF,ALT,AC,af,AN,DP,GT,AD,GQ,len_AD,REF_AD,ALT_AD,gt0,gt1
470,MTB_anc,334568,.,C,"G,*",2.0,0.0,1.0,8.0,1/1,"0,8,0",24.0,3,0,8,1,1
471,MTB_anc,334577,.,A,"G,*",1.0,1.0,0.5,9.0,1/2,"0,8,1",18.0,3,0,8,1,2
542,MTB_anc,336707,.,G,"A,*",2.0,0.0,1.0,12.0,1|1,"0,12,0",42.0,3,0,12,1,1
543,MTB_anc,336708,.,T,"C,*",2.0,0.0,1.0,13.0,1|1,"0,13,0",39.0,3,0,13,1,1
544,MTB_anc,336712,.,T,"C,*",2.0,0.0,1.0,13.0,1|1,"0,13,0",42.0,3,0,13,1,1
747,MTB_anc,467504,.,G,"A,*",2.0,0.0,1.0,27.0,1|1,"0,27,0",84.0,3,0,27,1,1
1174,MTB_anc,672504,.,C,"*,T,G",1.0,0.0,0.0,18.0,0|1,"13,5,0,0",99.0,4,13,5,0,1
1446,MTB_anc,814810,.,T,"C,G",0.0,2.0,0.0,50.0,2/2,"0,0,50",99.0,3,0,50,2,2
1653,MTB_anc,929094,.,C,"A,*",2.0,0.0,1.0,9.0,1/1,"0,9,0",27.0,3,0,9,1,1
1739,MTB_anc,976897,.,G,"*,T",2.0,0.0,1.0,34.0,1/1,"0,33,0",99.0,3,0,33,1,1


In [344]:
gen[gen.POS.isin(polym_list)][['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'AC', 'af', 'AN', 'DP','len_AD', 'REF_AD', 'ALT_AD',
       'gt0', 'gt1', 'dp', 'aF', 'AF']]

,#CHROM,POS,ID,REF,ALT,QUAL,AC,af,AN,DP,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF


In [350]:
gen[gen.ALT == '*']

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
233,MTB_anc,672504,.,C,*,3627.62,PASS,"AC=1,0,0;AF=0.500,0.00,0.00;AN=2;BaseQRankSum=...",GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:13,5,0,0:18:99:0|1:672481_T_A:137,0,3810,1...",1.0,0.0,0.0,18.0,0.000,2.0,1.11,18.000,0.4900,59.99,0|1,"13,5,0,0",99.0,"137,0,3810,176,3825,4002,176,3825,4002,4002",0.5,0.1751,0.3628,0|1,672481_T_A,672481,2,13.0,5.0,0,1,18.0,0.722222,0.277778
296,MTB_anc,976897,.,G,*,11694.04,PASS,"AC=2,0;AF=1.00,0.00;AN=2;BaseQRankSum=-1.260e-...",GT:AD:DP:GQ:PL,"1/1:0,33,0:34:99:1410,108,0,1271,107,1205",2.0,0.0,1.0,34.0,-0.126,34.0,0.0000,3.350,60.0000,10.54,1/1,"0,33,0",99.0,"1410,108,0,1271,107,1205",0.0,0.7664,0.0000,NaN,NaN,NaN,2,0.0,33.0,1,1,33.0,0.000000,1.000000
474,MTB_anc,1618637,.,C,*,1896.52,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=0.11...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,0.1113,14.591,0.3191,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
475,MTB_anc,1618640,.,C,*,1901.83,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=0.11...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,0.1113,14.591,0.3149,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
476,MTB_anc,1618650,.,G,*,1923.34,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=1.26...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,1.2682,14.591,0.1844,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
477,MTB_anc,1618652,.,C,*,1911.77,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=0.11...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,0.1113,14.591,0.2842,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
478,MTB_anc,1618653,.,G,*,1925.34,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=1.26...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,1.2682,14.591,0.1844,59.96,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
479,MTB_anc,1618656,.,G,*,1856.19,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=0.63...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,0.6386,14.253,0.2076,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
480,MTB_anc,1618663,.,C,*,1911.77,PASS,"AC=1,0;AF=0.500,0.00;AN=2;DP=15;ExcessHet=0.25...",GT:AD:DP:GQ:PL,"0/1:10,3,0:15:97:97,0,3393,134,3367,3590",1.0,0.0,0.5,15.0,2.000,15.0,0.2514,14.591,0.2587,59.92,0/1,"10,3,0",97.0,"97,0,3393,134,3367,3590",NaN,NaN,NaN,NaN,NaN,NaN,2,10.0,3.0,0,1,13.0,0.769231,0.230769
617,MTB_anc,2165503,.,T,*,12533.77,PASS,"AC=0,1;AF=0.00,0.500;AN=2;BaseQRankSum=1.05;DP...",GT:AD:DP:GQ:PGT:PID:PL:PS,"0|2:7,0,33:40:99:0|1:2165427_TTGCCGG_T:973,997...",0.0,1.0,0.0,40.0,1.050,40.0,0.0009,1.930,53.4300,14.29,0|2,"7,0,33",99.0,"973,997,2784,0,1788,1689",0.5,0.5251,0.2480,0|1,2165427_TTGCCGG_T,2165427,2,7.0,33.0,0,2,40.0,0.175000,0.825000


# INDELS

In [414]:
indel = import_VCF42_to_pandas(indel_file)

In [421]:
indel.tail()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
1550,MTB_anc,4321346,.,CG,C,3233.44,PASS,AC=2;AF=1.00;AN=2;DP=91;ExcessHet=0.7918;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,91:91:99:3249,274,0",2.0,1.0,2.0,91.0,0.7918,0.0,60.0,31.05,0.807,1/1,"0,91",99.0,"3249,274,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,91.0,1,1,91.0,0.0,1.0
1551,MTB_anc,4336090,.,A,AT,5226.48,PASS,AC=2;AF=1.00;AN=2;DP=131;ExcessHet=0.7918;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,131:131:99:5242,394,0",2.0,1.0,2.0,131.0,0.7918,0.0,60.0,34.07,0.838,1/1,"0,131",99.0,"5242,394,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,131.0,1,1,131.0,0.0,1.0
1552,MTB_anc,4338595,.,GT,G,8584.09,PASS,AC=2;AF=1.00;AN=2;DP=154;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,154:154:99:5755,464,0",2.0,1.0,2.0,154.0,3.0103,0.0,60.0,34.39,0.943,1/1,"0,154",99.0,"5755,464,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,154.0,1,1,154.0,0.0,1.0
1553,MTB_anc,4383144,.,C,CCGGGG,6037.13,PASS,AC=2;AF=1.00;AN=2;DP=90;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,90:90:99:4082,274,0",2.0,1.0,2.0,90.0,3.0103,0.0,60.0,31.08,0.921,1/1,"0,90",99.0,"4082,274,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,90.0,1,1,90.0,0.0,1.0
1554,MTB_anc,4400660,.,AC,A,6352.09,PASS,AC=2;AF=1.00;AN=2;DP=121;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,121:121:99:4335,364,0",2.0,1.0,2.0,121.0,3.0103,0.0,60.0,32.66,0.811,1/1,"0,121",99.0,"4335,364,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,121.0,1,1,121.0,0.0,1.0


In [417]:
indel[~indel.ALT.str.contains(r'(A|T|C|G)')]

/home/laura/env36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
909,MTB_anc,2960249,.,C,*,199.52,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=17;ExcessHet=3.010...",GT:AD:DP:GQ:PGT:PID:PL:PS,"2|2:0,0,17:17:51:1|1:2960237_CGCG_C:719,719,71...",0.0,2.0,0.0,17.0,2.0,17.0,3.0103,0.0,60.00,2|2,"0,0,17",51.0,"719,719,719,51,51,0",NaN,NaN,NaN,1|1,2960237_CGCG_C,2960237,2,0.0,17.0,2,2,17.0,0.0,1.0
967,MTB_anc,3119735,.,A,*,1117.55,PASS,"AC=0,2;AF=0.00,1.00;AN=2;BaseQRankSum=2.30;DP=...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.3,22.0,3.0103,60.0,19.61,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",1.0,3.076,0.0,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0
968,MTB_anc,3119741,.,T,*,1619.57,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=22;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.0,22.0,3.0103,0.0,60.00,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0
1485,MTB_anc,3119738,.,GGT,*,1661.57,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=22;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.0,22.0,3.0103,0.0,60.00,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0


In [164]:
df = import_VCF42_to_pandas(file)

In [92]:
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
0,MTB_anc,1277,.,C,A,12726.43,PASS,AC=2;AF=1.00;AN=2;DP=187;ExcessHet=0.0006;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,187:187:99:7003,561,0",2.0,1.0,2.0,187.0,0.0006,0.0,1.0000,60.0,25.36,0.791,1/1,"0,187",99.0,"7003,561,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,187.0,1,1,187.0,0.0,1.0
1,MTB_anc,2532,.,C,T,71031.72,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.83;DP=86;Exce...,GT:AD:DP:GQ:PL,"1/1:0,86:86:99:2878,257,0",2.0,1.0,2.0,86.0,3.0103,0.0,-0.0000,60.0,34.33,0.159,1/1,"0,86",99.0,"2878,257,0",1.830,0.0,0.316,NaN,NaN,NaN,2,0.0,86.0,1,1,86.0,0.0,1.0
2,MTB_anc,9143,.,C,T,128501.72,PASS,AC=2;AF=1.00;AN=2;DP=57;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,57:57:99:2071,170,0",2.0,1.0,2.0,57.0,3.0103,0.0,-0.0000,60.0,25.00,0.956,1/1,"0,57",99.0,"2071,170,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,57.0,1,1,57.0,0.0,1.0
3,MTB_anc,13460,.,G,A,129292.72,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.78;DP=45;Exce...,GT:AD:DP:GQ:PL,"1/1:0,45:45:99:1791,135,0",2.0,1.0,2.0,45.0,3.0103,0.0,-0.0000,60.0,34.57,0.307,1/1,"0,45",99.0,"1791,135,0",1.780,0.0,0.919,NaN,NaN,NaN,2,0.0,45.0,1,1,45.0,0.0,1.0
4,MTB_anc,14251,.,G,A,68528.18,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.716e+00;DP=3...,GT:AD:DP:GQ:PL,"1/1:0,33:33:98:1174,98,0",2.0,1.0,2.0,33.0,0.0000,0.0,1.0000,60.0,33.31,1.286,1/1,"0,33",98.0,"1174,98,0",-1.716,0.0,0.067,NaN,NaN,NaN,2,0.0,33.0,1,1,33.0,0.0,1.0


In [93]:
df[(df.POS == 2626189) | (df.POS == 2626191)]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
695,MTB_anc,2626189,.,A,C,101892.61,PASS,"AC=1,1;AF=0.500,0.500;AN=2;BaseQRankSum=0.789;...",GT:AD:DP:GQ:PL,"1/2:0,29,5:34:99:1425,207,120,1218,0,1245",1.0,1.0,0.5,34.0,0.789,34.0,3.0103,0.0,45.58,29.66,1/2,"0,29,5",99.0,"1425,207,120,1218,0,1245",0.5,0.0160,-0.041,NaN,NaN,NaN,2,0.0,29.0,1,2,29.0,0.0,1.0
696,MTB_anc,2626191,.,T,C,101887.04,PASS,"AC=1,1;AF=0.500,0.500;AN=2;BaseQRankSum=0.330;...",GT:AD:DP:GQ:PL,"1/2:0,29,5:34:99:1425,207,120,1218,0,1245",1.0,1.0,0.5,34.0,0.330,34.0,3.0103,0.0,45.49,36.86,1/2,"0,29,5",99.0,"1425,207,120,1218,0,1245",0.5,0.0064,-0.041,NaN,NaN,NaN,2,0.0,29.0,1,2,29.0,0.0,1.0


In [64]:
df.columns

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'sample', 'AC', 'af', 'AN', 'DP', 'ExcessHet', 'FS', 'InbreedingCoeff',
       'MQ', 'QD', 'SOR', 'GT', 'AD', 'GQ', 'PL', 'BaseQRankSum', 'MQRankSum',
       'ReadPosRankSum', 'PGT', 'PID', 'PS', 'len_AD', 'REF_AD', 'ALT_AD',
       'AD_list', 'gt0', 'gt1', 'dp', 'aF', 'AF'],
      dtype='object')

In [66]:
for i in df.AD_list.values:
    if len(i) > 1:
        print(i)
        print(max(i))

[29, 5]
29
[29, 5]
29


In [97]:
add_window_distance(df)

In [114]:
df.index[df["POS"] == 1480205][0]

400

In [129]:
list_pos = df.POS.to_list() #all positions
set_pos = set(list_pos) #to set for later comparing
max_pos = max(df.POS.to_list()) #max to iter over positions (independent from reference)

all_list = list(range(1, max_pos + 1)) #create a list to slide one by one
    
df_header = "window_" + str(30)

df[df_header] = 1
   
#Slide over windows
for i in range(0,max_pos,1):
    window_pos = all_list[i:i+30] #
    
    set_window_pos = set(window_pos)
    
    #How many known positions are in every window for later clasification
    num_conglomerate = set_pos & set_window_pos
    
    if len(num_conglomerate) > 1:
        for i in num_conglomerate:
            #print(i, len(num_conglomerate))
            index = df.index[df["POS"] == i][0]
            if df.loc[index,df_header] < len(num_conglomerate):
                df.loc[index,df_header] = len(num_conglomerate)

In [132]:
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF,window_10,window_20,window_30
0,MTB_anc,1277,.,C,A,12726.43,PASS,AC=2;AF=1.00;AN=2;DP=187;ExcessHet=0.0006;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,187:187:99:7003,561,0",2.0,1.0,2.0,187.0,0.0006,0.0,1.0000,60.0,25.36,0.791,1/1,"0,187",99.0,"7003,561,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,187.0,1,1,187.0,0.0,1.0,1,1,1
1,MTB_anc,2532,.,C,T,71031.72,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.83;DP=86;Exce...,GT:AD:DP:GQ:PL,"1/1:0,86:86:99:2878,257,0",2.0,1.0,2.0,86.0,3.0103,0.0,-0.0000,60.0,34.33,0.159,1/1,"0,86",99.0,"2878,257,0",1.830,0.0,0.316,NaN,NaN,NaN,2,0.0,86.0,1,1,86.0,0.0,1.0,1,1,1
2,MTB_anc,9143,.,C,T,128501.72,PASS,AC=2;AF=1.00;AN=2;DP=57;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,57:57:99:2071,170,0",2.0,1.0,2.0,57.0,3.0103,0.0,-0.0000,60.0,25.00,0.956,1/1,"0,57",99.0,"2071,170,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,57.0,1,1,57.0,0.0,1.0,1,1,1
3,MTB_anc,13460,.,G,A,129292.72,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.78;DP=45;Exce...,GT:AD:DP:GQ:PL,"1/1:0,45:45:99:1791,135,0",2.0,1.0,2.0,45.0,3.0103,0.0,-0.0000,60.0,34.57,0.307,1/1,"0,45",99.0,"1791,135,0",1.780,0.0,0.919,NaN,NaN,NaN,2,0.0,45.0,1,1,45.0,0.0,1.0,1,1,1
4,MTB_anc,14251,.,G,A,68528.18,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.716e+00;DP=3...,GT:AD:DP:GQ:PL,"1/1:0,33:33:98:1174,98,0",2.0,1.0,2.0,33.0,0.0000,0.0,1.0000,60.0,33.31,1.286,1/1,"0,33",98.0,"1174,98,0",-1.716,0.0,0.067,NaN,NaN,NaN,2,0.0,33.0,1,1,33.0,0.0,1.0,1,1,1


In [131]:
df[df.window_20 > 6]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,InbreedingCoeff,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF,window_10,window_20,window_30
394,MTB_anc,1480186,.,C,T,6048.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.507e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:59,10:69:99:0|1:1480174_C_G:242,0,2484:148...",1.0,0.5,2.0,69.0,15.9739,13.251,-0.2270,59.41,7.78,0.725,0|1,"59,10",99.0,"242,0,2484",-2.507,-6.097,1.340,0|1,1480174_C_G,1480174,2,59.0,10.0,0,1,69.0,0.855072,0.144928,3,7,9
395,MTB_anc,1480189,.,C,G,5901.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-7.780e-01;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:60,10:70:99:0|1:1480174_C_G:239,0,2526:148...",1.0,0.5,2.0,70.0,15.9739,12.064,-0.2270,59.41,7.55,0.709,0|1,"60,10",99.0,"239,0,2526",-0.778,-6.097,1.540,0|1,1480174_C_G,1480174,2,60.0,10.0,0,1,70.0,0.857143,0.142857,4,7,9
396,MTB_anc,1480195,.,G,T,5712.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.656e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:58,10:68:99:0|1:1480174_C_G:239,0,2526:148...",1.0,0.5,2.0,68.0,15.9739,10.545,-0.2270,59.42,7.34,0.692,0|1,"58,10",99.0,"239,0,2526",-2.656,-6.181,1.470,0|1,1480174_C_G,1480174,2,58.0,10.0,0,1,68.0,0.852941,0.147059,4,7,9
397,MTB_anc,1480197,.,G,C,5559.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-1.674e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:59,10:69:99:0|1:1480174_C_G:233,0,2610:148...",1.0,0.5,2.0,69.0,15.9739,7.754,-0.2270,59.45,6.98,0.779,0|1,"59,10",99.0,"233,0,2610",-1.674,-6.231,1.590,0|1,1480174_C_G,1480174,2,59.0,10.0,0,1,69.0,0.855072,0.144928,4,7,9
398,MTB_anc,1480198,.,T,C,5490.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=2.07;DP=71;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:61,10:71:99:0|1:1480174_C_G:227,0,2694:148...",1.0,0.5,2.0,71.0,15.9739,7.643,-0.2270,59.46,6.85,0.802,0|1,"61,10",99.0,"227,0,2694",2.070,-6.320,1.610,0|1,1480174_C_G,1480174,2,61.0,10.0,0,1,71.0,0.859155,0.140845,4,7,9
399,MTB_anc,1480204,.,C,T,5172.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.933e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:61,9:70:99:0|1:1480174_C_G:233,0,2604:1480174",1.0,0.5,2.0,70.0,15.9739,9.441,-0.2270,59.49,6.55,0.997,0|1,"61,9",99.0,"233,0,2604",-2.933,-6.210,1.350,0|1,1480174_C_G,1480174,2,61.0,9.0,0,1,70.0,0.871429,0.128571,4,7,9
400,MTB_anc,1480205,.,G,A,4923.81,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.230e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:62,9:71:99:0|1:1480174_C_G:233,0,2604:1480174",1.0,0.5,2.0,71.0,15.9739,8.302,-0.2270,59.52,6.22,1.092,0|1,"62,9",99.0,"233,0,2604",-2.230,-6.205,1.410,0|1,1480174_C_G,1480174,2,62.0,9.0,0,1,71.0,0.873239,0.126761,4,7,9
700,MTB_anc,2631929,.,C,G,16187.76,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.621;DP=79;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:48,31:79:99:0|1:2631929_C_G:1157,0,4997:26...",1.0,0.5,2.0,79.0,32.8523,3.120,-0.4345,57.05,6.68,1.016,0|1,"48,31",99.0,"1157,0,4997",0.621,-6.656,-0.888,0|1,2631929_C_G,2631929,2,48.0,31.0,0,1,79.0,0.607595,0.392405,4,7,7
701,MTB_anc,2631931,.,G,A,16730.76,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.400e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:48,31:79:99:0|1:2631929_C_G:1199,0,4994:26...",1.0,0.5,2.0,79.0,32.8523,2.177,-0.4345,57.05,6.95,0.952,0|1,"48,31",99.0,"1199,0,4994",-2.400,-6.435,-0.750,0|1,2631929_C_G,2631929,2,48.0,31.0,0,1,79.0,0.607595,0.392405,4,7,7
702,MTB_anc,2631932,.,C,G,16726.76,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-9.900e-01;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:48,31:79:99:0|1:2631929_C_G:1199,0,4994:26...",1.0,0.5,2.0,79.0,32.8523,4.089,-0.4345,57.09,6.84,0.990,0|1,"48,31",99.0,"1199,0,4994",-0.990,-6.418,-0.563,0|1,2631929_C_G,2631929,2,48.0,31.0,0,1,79.0,0.607595,0.392405,4,7,7


In [193]:
def bed_to_dict(bed_file):
    dict_range_positions = {}
    with open(bed_file, 'r') as f:
        for line_number, line in enumerate(f):
            line_split = line.split(None) #This split by any blank character
            start = line_split[1]
            end = line_split[2]
            if len(line_split) == 3 and start.isdigit() and end.isdigit():
                start = int(start)
                end = int(end)
                dict_range_positions[start] = end
            else:
                if line_number != 0:
                    print("This file is not in bed format")
                    sys.exit(1)
    return dict_range_positions

In [194]:
dict_test = bed_to_dict(bed)

In [220]:
def annotate_bed(bed_file, vcf_df, tag):
    dict_position = bed_to_dict(bed_file)
    all_positions = vcf_df.POS.tolist()
    for start, end in dict_position.items():
        for pos in all_positions:
            index = vcf_df.index[vcf_df["POS"] == pos][0]
            if pos >= start and pos <= end:
                vcf_df.loc[index, tag] = True
            else:
                vcf_df.loc[index, tag] = False


In [262]:
def annotate_bed(bed_file, pos):
    dict_position = bed_to_dict(bed_file)
    #print(dict_position)
    for start, end in dict_position.items():
        print(start, ":", end)
        if pos >= start and pos <= end:
            print(pos)
            return True
        else:
            return False
#Des not work bedcause function ends with the first return

In [278]:
def annotate_bed(bed_file, pos):
    """
    Identify a position within a range
    credits: https://stackoverflow.com/questions/6053974/python-efficiently-check-if-integer-is-within-many-ranges
    """
    dict_position = bed_to_dict(bed_file)
    if any(start <= pos <= end for (start, end) in dict_position.items()):
        return True
    else:
        return False


In [279]:
annotate_bed(bed, 149532)

False

In [280]:
#df['Is_repeat'] = df.apply(annotate_bed(), axis=1, args=(bed_file=bed))

In [274]:
df['Is_repeat'] = df['POS'].apply(lambda x: annotate_bed(bed,x))

In [281]:
df[df.Is_repeat == True].shape

(173, 40)

In [96]:
def add_window_distance(vcf_df, window_size=10):
    """
    Add a column indicating the maximum number of SNPs in a windows of 10
    """
    list_pos = vcf_df.POS.to_list() #all positions
    set_pos = set(list_pos) #to set for later comparing
    max_pos = max(vcf_df.POS.to_list()) #max to iter over positions (independent from reference)

    all_list = list(range(1, max_pos + 1)) #create a list to slide one by one
    
    df_header = "window_" + str(window_size)

    #Create sets
    set_2 = set()
    set_3 = set()
    set_4 = set()
    set_5 = set()
    
    sets = [set_2, set_3, set_4, set_5]
    
    #Slide over windows
    for i in range(0,max_pos,1):
        window_pos = all_list[i:i+window_size]
        set_window_pos = set(window_pos)
        #How many known positions are in every window for later clasification
        num_conglomerate = set_pos & set_window_pos
        
        if len(num_conglomerate) > 4:
            set_5.update(num_conglomerate)
        elif len(num_conglomerate) == 4:
            set_4.update(num_conglomerate)
        elif len(num_conglomerate) == 3:
            set_3.update(num_conglomerate)
        elif len(num_conglomerate) == 2:
            set_2.update(num_conglomerate)
            
    #Remove positions in a higher number of sets
    for set_num in range(0, len(sets)):
        if set_num < (len(sets) - 1):
            sets[set_num] = sets[set_num] - sets[set_num + 1]
            
    for index, _ in vcf_df.iterrows():
        if vcf_df.loc[index,'POS'] in sets[0]:
            vcf_df.loc[index, df_header] = 2
        elif vcf_df.loc[index,'POS'] in sets[1]:
            vcf_df.loc[index, df_header] = 3
        elif vcf_df.loc[index,'POS'] in sets[2]:
            vcf_df.loc[index, df_header] = 4
        elif vcf_df.loc[index,'POS'] in sets[3]:
            vcf_df.loc[index, df_header] = 5
        else:
            vcf_df.loc[index, df_header] = 1
            
    vcf_df[df_header] = vcf_df[df_header].astype(int)